# Notebook 4: Quantitative scintigraphy image analysis

In [0]:
!git clone https://github.com/albertine/RPM_IBM_Module_IA.git

In [0]:
# Import required packages
import numpy as np
import matplotlib.pyplot as plt
import os

# Install and import SimpleITK
!pip install --upgrade SimpleITK
import SimpleITK as sitk

# Import some interactive IPython widgets
# The interact function (ipywidgets.interact) automatically creates user interface (UI) controls for exploring code and data interactively.
from ipywidgets import interact, interactive, fixed

# Define current working directory
PWD_DIR = os.getcwd()
# Define output data directory
OUTPUT_DIR = os.path.join(PWD_DIR,'output')
# If it doesn't exist, create it
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## 1. Cortical renal scintigraphy using <sup>99m</sup>Tc-DMSA

### Background
The renal scintigraphy using <sup>99m</sup>Tc-DMSA constitutes a non-invasive and functional method that is of appreciable interest in the qualitative study of renal parenchyma and the evaluation of the separate renal function.

After <sup>99m</sup>Tc-DMSA is injected intravenously it slowly concentrates in the kidney where it mostly becomes bound to the proximal tubular cells. There is a low extraction efficiency by the kidneys, but 2 or 3 hours following injection there is sufficient uptake to obtain good images of the functioning renal cortex with low background.

### Data analysis
Usually, a percent differential renal function (DRF) is calculated from the posterior view by drawing regions of interest (ROI) around each kidney. A
background ROI should also be drawn in any nearby non-renal area. The
position of the background ROI makes very little difference except with very
poorly functioning kidneys. Counts in the background region are used to
subtract background counts from each kidney ROI (scaled for the relative
size of the ROIs).

DRF of each kidney is then calculated from the
percentage of background subtracted counts in each kidney as follows:

\begin{equation}
DRF_{right}=100\times \frac{Right\ kidney\ counts -background\ counts}{(Right\ kidney\ counts -background\ counts)+(Left\ kidney\ counts -background\ counts)}
\end{equation}


In the following sections, let's see how to apply a semi-automatic segmentation method on <sup>99m</sup>Tc-DMSA images so as to calculate DRF of each kidney

### Pre-processing: data exploration

In [0]:
# Change directory to folder /data/rein
os.chdir('/content/RPM_IBM_Module_IA/data/rein/')

In [0]:
# Display all files stored in folder /data/rein
whole_list = [file for file in os.listdir()]
dcm_list   = [file for file in os.listdir() if file.endswith(".dcm")]
print("Entire list: ", whole_list)
print("Lis of .dcm files: ", dcm_list)

In [0]:
# Read the image corresponding to posterior view of 99m DMSA static scintigraphy 
dmsa_post=sitk.ReadImage('PosteriorStatic001_DS.dcm')
# Image pixel type
print(dmsa_post.GetPixelIDTypeAsString())

In [0]:
# Import pydicom package for inspecting and modifying DICOM files in an easy pythonic way
!pip install --upgrade pydicom
import pydicom

# Read DICOM information
ds=pydicom.filereader.dcmread('PosteriorStatic001_DS.dcm')

# Print several useful DICOM dataset information 
print("Exam modality: ", ds.Modality)
print("Image attributes - lines/rows (in pixels)")
print (ds[0x0028,0x0010])
print (ds[0x0028,0x0011])
print("Patient information (name, weight)")
print (ds[0x0010,0x0010])
print (ds[0x0008,0x1030])
print("Acquisition information")
print (ds[0x0008,0x1010])
print (ds[0x0008,0x1030])
print (ds[0x0009,0x1010])
print (ds[0x0010,0x1030])
print (ds[0x0010,0x0030])
print(ds[0x0018,0x1242])
print(ds[0x0054,0x0022][0][0x0018,0x1180])
print(ds[0x0054,0x0022][0][0x0018,0x1181])
# If you want to extract and display a specific DICOM tag value 
print("For instance, to only extract the Study Name value: ",ds[0x0009,0x1010].value)
# If you want to print all DICOM tags
#print(ds)

In [0]:
# Access to posterior view of 99m DMSA static scintigraphy image attributes
print("Number of pixels: ",dmsa_post.GetNumberOfPixels())
print("Image size: ",dmsa_post.GetSize())
print("Image spacing (voxel size): ",dmsa_post.GetSpacing())
print("Image details: ",dmsa_post.GetWidth(),dmsa_post.GetHeight(),dmsa_post.GetDepth())
print("Image dimensions: ",dmsa_post.GetDimension())
print("Image depth (number of components): ",dmsa_post.GetNumberOfComponentsPerPixel())

In [0]:
# Access to posterior view of 99m DMSA static scintigraphy image basic statistics
stats = sitk.StatisticsImageFilter()
stats.Execute(dmsa_post)
print(stats)
# Print only minimum and maximum intensity values
print("Image minimum intensity: ", stats.GetMinimum())
print("Image maximum intensity: ", stats.GetMaximum())

In [0]:
# Conversion of posterior view image to NumPy array
dmsa_post_arr = sitk.GetArrayFromImage(dmsa_post)
print(dmsa_post_arr.shape)
# Remove the third dimension using reshape function
dmsa_post_arr = dmsa_post_arr.reshape(dmsa_post.GetWidth(),dmsa_post.GetHeight())
print(dmsa_post_arr.shape)

In [0]:
# Display posterior view image using different color maps
cpt = 1
color_list = ['viridis','jet','binary','gray']
plt.subplots(2, 2, figsize=(10, 10)) # we force a 10x10 size for display
for color in color_list:
    plt.subplot((len(color_list))/2,2,cpt)
    plt.imshow(dmsa_post_arr,cmap=color)
    plt.title(color)
    plt.axis('off')
    cpt+=1

In [0]:
# Plot the image intensity histogram using different number of bins and range values
plt.subplots(2, 2, figsize=(10, 10))
plt.subplot(2,2,1)

# Hist method computes and draws the histogram of x. 
# The return value is a tuple (n, bins, patches) or ([n0, n1, ...], bins, [patches0, patches1,...]) if the input contains multiple data 
# Flatten method returns a copy of the array collapsed into one dimension
plt.hist(dmsa_post_arr.flatten(),bins=256,range=(stats.GetMinimum(),stats.GetMaximum()))
plt.title("256 bins, range 0-512")
plt.subplot(2,2,2)
plt.hist(dmsa_post_arr.flatten(),256,range=(0,256))
plt.title("256 bins, range 0-256")
plt.subplot(2,2,3)
plt.hist(dmsa_post_arr.flatten(),64,[0,256])
plt.title("64 bins, range 0-256")
plt.subplot(2,2,4)
plt.hist(dmsa_post_arr.flatten(),32,[0,64])
plt.title("32 bins, range 0-64")
plt.suptitle('Histograms',fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

In [0]:
# Let's see how to perform a basic thresholding of 99m DMSA static scintigraphy image attributes
# Based on this previous histogram analysis, we can take 80 as threshold
print("Maximum intensity value in image is",dmsa_post_arr.max(),'\n')
dmsa_post_bin_arr = dmsa_post_arr>80
print(dmsa_post_bin_arr)
# dmsa_post_bin_arr only contains booleans (true or false) for each pixel
# for each pixel, dmsa_post_bin_arr answers the question is value > threshold ?
# if value > threshold then condition is true
# if value < threshold then condition is false

In [0]:
# We can use plt.imshow because Python considers two intensity values
# One for True and another one for False
plt.imshow(dmsa_post_bin_arr)

In [0]:
# Overlay of the two images (dmsa_post and its mask corresponding to a threshold of 80)
plt.imshow(dmsa_post_arr)
dmsa_post_bin_arr_mask=np.ma.masked_where(dmsa_post_bin_arr==True, dmsa_post_bin_arr)
plt.imshow(dmsa_post_bin_arr_mask, alpha=0.7)

In [0]:
# Let's try several absolute threshold values
thresh_abs_list=[20,30,40,50,60,80]
cpt = 1
plt.subplots(2,3,figsize=(15,10))
for thresh_abs in thresh_abs_list:
    plt.subplot(2,3,cpt)
    plt.imshow(dmsa_post_arr>thresh_abs)
    plt.title("threshold value : "+str(thresh_abs))
    plt.axis('off')
    cpt+=1

In [0]:
# Let's try several relative threshold values
thresh_rel_list=[.05,.1,.3,.5,.6,.8]
cpt = 1
plt.subplots(2,3,figsize=(15,10))
for thresh_rel in thresh_rel_list:
    plt.subplot(2,3,cpt)
    plt.imshow(dmsa_post_arr>(dmsa_post_arr.max()*thresh_rel))
    plt.title("Relative threshold: "+str(thresh_rel))
    plt.axis('off')
    cpt+=1

In [0]:
# Source image to be processed: dmsa_post (SimpleITK image) or dmsa_post_arr (NumPy array)
# We apply a mask corresponding to the thresholding binarisation previously obtained
# Arbitrarily, we'll take a relative threshold value of 50% maximum intensity value
thresh50 = 0.5 * dmsa_post_arr.max()
mask50 = (dmsa_post_arr > thresh50) 
dmsa_post50 = dmsa_post_arr * thresh50 # set each pixel outside the mask to zero (because False)
plt.imshow(dmsa_post50)

In [0]:
# Apply this mask for different relative threshold values but now keep the intensity values
# Let's try several relative threshold values
thresh_rel_list=[.05,.1,.2,.3,.5,.7]
cpt = 1
plt.subplots(2,3,figsize=(15,10))
for thresh_rel in thresh_rel_list:
    plt.subplot(2,3,cpt)
    plt.imshow(dmsa_post_arr*(dmsa_post_arr>(dmsa_post_arr.max()*thresh_rel)))
    plt.title("Relative threshold: "+str(thresh_rel))
    plt.axis('off')
    cpt+=1

Based on this data exploration step, a relative threshold of 20% seems to be a good value to perform a semi-automatic segmentation of each kidney

### Quantitative analysis

**First step**: we can divide the image in two along horizontal direction (using a vertical separation...)

Hence, we obtain two posterior view 99m Tc DMSA scintigraphy images, one for right kidney and another one for left kidney

In [0]:
width_dmsa = dmsa_post.GetWidth()

Lkidney=dmsa_post_arr[:,0:int(width_dmsa/2)]
Rkidney=dmsa_post_arr[:,int(width_dmsa/2):width_dmsa]

plt.subplots(1,2,figsize=(5,5))
plt.subplot(1,2,1)
plt.imshow(Lkidney)
plt.subplot(1,2,2)
plt.imshow(Rkidney)

**Second step**: quantitative analysis in each ROI (Right and Left) using Pandas package

**What's Pandas for?**

This tool is essentially your data’s home. Through pandas, you get acquainted with your data by cleaning, transforming, and analyzing it.

For example, say you want to explore a dataset stored in a CSV on your computer. Pandas will extract the data from that CSV into a DataFrame — a table, basically — then let you do things like:

* Calculate statistics and answer questions about the data, like
 * What's the average, median, max, or min of each column?
 * Does column A correlate with column B?
 * What does the distribution of data in column C look like?
* Clean the data by doing things like removing missing values and filtering rows or columns by some criteria
* Visualize the data with help from Matplotlib. Plot bars, lines, histograms, bubbles, and more.
* Store the cleaned, transformed data back into a CSV, other file or database

Before you jump into the modeling or the complex visualizations you need to have a good understanding of the nature of your dataset and pandas is the best avenue through which to do that.

In [0]:
import pandas as pd

imgLkidney=sitk.GetImageFromArray(Lkidney)
imgLkidney.SetSpacing([dmsa_post.GetSpacing()[0], dmsa_post.GetSpacing()[1],1])
imgRkidney=sitk.GetImageFromArray(Rkidney)
imgRkidney.SetSpacing([dmsa_post.GetSpacing()[0], dmsa_post.GetSpacing()[1],1])
imgTotal=sitk.GetImageFromArray(dmsa_post_arr)

# Let's use 20% as a relative threshold to semi-automatically draw ROI around each kidney and 5% for background ROI
thresh20 = 0.2 * dmsa_post_arr.max()
thresh5 = 0.05 * dmsa_post_arr.max()
segL=sitk.BinaryThreshold(imgLkidney, upperThreshold=thresh20, insideValue=0, outsideValue=1)
segR=sitk.BinaryThreshold(imgRkidney, upperThreshold=thresh20, insideValue=0, outsideValue=1)
segback=sitk.BinaryThreshold(imgTotal, lowerThreshold=thresh5, upperThreshold=thresh20, insideValue=1, outsideValue=0)

# If you want, you can write all the ROIs stored as a nii (or mha) files  
#output_file_name = os.path.join(OUTPUT_DIR, 'segback.nii') 
#sitk.WriteImage(segback, output_file_name)

# LabelIntensityStatisticsImageFilter is a convenient class to convert a label image to a label map and valuate the statistics attributes at once
intensity_stats=sitk.LabelIntensityStatisticsImageFilter()

# Home-made function to compute and display standard statistics derived from a specific ROI
def show_stats(seg,img,file):
    intensity_stats=sitk.LabelIntensityStatisticsImageFilter()
    intensity_stats.Execute(seg,img)
    for i in intensity_stats.GetLabels():
        #print(intensity_stats.GetMean(i), intensity_stats.GetPhysicalSize(i))
        stats_list={"Volume (mm^3)": [intensity_stats.GetPhysicalSize(i)],
                    "Volume (number of pixels)": [intensity_stats.GetNumberOfPixels(i)],
                    "Intensity mean": [intensity_stats.GetMean(i)],
                    "Intensity standard deviation": [intensity_stats.GetStandardDeviation(i)],
                    "Intensity max": [intensity_stats.GetMaximum(i)],
                    "Intensity min": [intensity_stats.GetMinimum(i)],
                    "Intensity median": [intensity_stats.GetMedian(i)],
                    }
    stats=pd.DataFrame(stats_list, index=[intensity_stats.GetLabels()[0]])
    stats.to_csv(file, sep='\t')
    return stats

In [0]:
# Show stats in left kidney
interact(show_stats, seg=fixed(segL), img=fixed(imgLkidney), file='Lkidney');

In [0]:
# Show stats in right kidney
interact(show_stats, seg=fixed(segR), img=fixed(imgRkidney), file='Rkidney');

In [0]:
# Show stats in background
interact(show_stats, seg=fixed(segback), img=fixed(imgTotal), file='background');

## Exercise 1
Derive from the above-displayed image statistics DRF of each kidney and check you obtain values between 45% and 55%, range representative of a normal renal function

A mentioned in the Renal Cortical Scintigraphy (DMSA scan) Clinical Guidelines published by the British nuclear medicine society, if an anterior view has also been obtained, background subtracted kidney
counts should also be calculated from this image. The geometric mean of
the posterior and anterior background subtracted counts in each kidney
should then be calculated.
\begin{equation}
Geometric\ Mean\ Count = \sqrt {Posterior\ Count \times Anterior Count}
\end{equation}
The geometric mean DRF should be calculated using these geometric mean
counts. This gives an approximate correction for the fact that kidneys may lie
at different depths. It is particularly important for ectopic or malrotated
kidneys.

## Exercise 2
Calculate geometric mean DRF of each kidney using geometric mean counts derived (both kidney and background) from the posterior and anterior views of <sup>99m</sup>Tc-DMSA scintigraphy

## 2. <sup>99m</sup>Tc-sestamibi and iodine 123 parathyroid scintigraphy

### Background
<sup>99m</sup>Tc-sestamibi is an isonitrile radionuclide imaging agent that, when used with subtraction iodine 123 thyroid scans, has the potential for imaging abnormal parathyroid glands.

Sodium iodine 123 is administered intravenously, followed 2 hours later by <sup>99m</sup>Tc-sestamibi. The patient is then positioned for imaging. The distribution images of <sup>99m</sup>Tc-sestamibi and <sup>123</sup>I are simultaneously recorded. Three to 5 minutes after <sup>99m</sup>Tc-sestamibi injection, an anterior view of the neck and mediastinum is acquired for 5 minutes to detect any ectopic uptake, from the angle of the mandible to the heart. The parallel-holes collimator is then replaced by a pinhole collimator and an anterior view of the thyroid region was obtained for 10 minutes. The <sup>123</sup>I image is subtracted from the <sup>99m</sup>Tc-sestamibi image. 

Given there will be more counts in the thyroid on the <sup>123</sup>I image, it must be matched to the <sup>99m</sup>Tc image before subtraction and a scaling factor is used so that the counts in the subtraction image are reduced by this factor, pixel by pixel.  The scaling factor is calculated
from the ratio of these two values. This adjusted
image is then subtracted from the 99mTc image
and the results displayed as a new image.  

In [0]:
# Change directory to /data/para
os.chdir('/content/RPM_IBM_Module_IA/data/para')

## Exercise 3

*   Display all files stored in folder /data/para
*   Read and display the PINHOLE_Tc99m001_DS.dcm image
*   Read and display corresponding DICOM information and image attributes
*   Find a specific DICOM tag relative to dual-tracer imaging technique
*   Separate and display image obtained for each tracer as well as their histograms
*   Compute the ratio between the <sup>99m</sup>Tc-sestamibi and the 123 iodine images (have a look at the cumulative sum function - cumsum - in NumPy) 
*   Apply this scaling factor to the 123 iodine image and substract it from <sup>99m</sup>Tc image



<a href="#ex3answer">Answer to exercise 3</a>

### Pre-processing: data exploration

In [0]:
# Display all files stored in folder /data/para
liste_complete = [file for file in os.listdir()]
liste_dcm      = [file for file in os.listdir() if file.endswith(".dcm")]
print("Entire list: ",liste_complete)
print("Lis of .dcm files: ",liste_dcm)

In [0]:
# Read and display image
para_dual=sitk.ReadImage('PINHOLE_Tc99m001_DS.dcm')
para_dual_arr=sitk.GetArrayFromImage(para_dual)
print(para_dual_arr.shape)
plt.subplots(1,2, figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(para_dual_arr[0])
plt.subplot(1,2,2)
plt.imshow(para_dual_arr[1])

In [0]:
# Read DICOM information
ds=pydicom.filereader.dcmread('PINHOLE_Tc99m001_DS.dcm')

# Print several useful DICOM dataset information 
print("Exam modality: ", ds.Modality)
print("Image attributes - lines/rows (in pixels)")
print (ds[0x0028,0x0010])
print (ds[0x0028,0x0011])
print("Patient information (name, weight)")
print (ds[0x0010,0x0010])
print (ds[0x0008,0x1030])
print("Acquisition information")
print (ds[0x0008,0x1010])
print (ds[0x0008,0x1030])
print (ds[0x0009,0x1010])
print (ds[0x0010,0x1030])
print (ds[0x0010,0x0030])
print(ds[0x0018,0x1242])
print(ds[0x0054,0x0022][0][0x0018,0x1180])
print(ds[0x0054,0x0022][0][0x0018,0x1181])
# If you want to extract and display a specific DICOM tag value 
print("For instance, to only extract the Study Name value: ",ds[0x0009,0x1010].value)
# Print all DICOM tags
#print(ds)

In [0]:
# Image attributes
print("Number of pixels: ",para_dual.GetNumberOfPixels())
print("Image size: ",para_dual.GetSize())
print("Image spacing (voxel size): ",para_dual.GetSpacing())
print("Image details: ",para_dual.GetWidth(),para_dual.GetHeight(),para_dual.GetDepth())
print("Image dimensions: ",para_dual.GetDimension())
print("Image depth (number of components): ",para_dual.GetNumberOfComponentsPerPixel())

In [0]:
# Specific but important other image attributes
print("Number of energy windows: ",ds.NumberOfEnergyWindows)
print("IMPORTANT: according to the following tag ")
print(ds[0x0011,0x100d])
print("the first image corresponds to the first tracer marked with 99mTc (MIBI-Tc)")
print("the second tracer is iodine-123")


### Using of dual-isotope subtraction method
**First step**: separation of the 2 images

In [0]:
# Using of dual-isotope subtraction method
print(sitk.GetArrayFromImage(para_dual).shape)
para_t1 = sitk.GetArrayFromImage(para_dual)[0]
para_t2 = sitk.GetArrayFromImage(para_dual)[1]
#para_t1 = para_t1.reshape(256,256)

In [0]:
plt.subplots(2,2,figsize=(10,10))
plt.subplot(2,2,1)
plt.title("99mTc-sestamibi")
plt.imshow(para_t1,aspect='auto')
plt.subplot(2,2,2)
plt.imshow(para_t2,aspect='auto')
plt.title("123 iodine")
plt.subplot(2,2,3)
plt.hist(para_t1.flatten(),200,[0,200]); plt.show
plt.subplot(2,2,4)
plt.hist(para_t2.flatten(),200,[0,200]); plt.show

**Second step**: computation of the ratio between the two images <sup>99m</sup>Tc-sestamibi and 123 iodine

In [0]:
# Sum of all counts received for the first tracer (99mTc-sestamibi)
# Cumsum function returns the cumulative sum of the elements along a given axis
para_t1_sum=para_t1.cumsum()[-1]
# and for the second tracer (123 I)
para_t2_sum=para_t2.cumsum()[-1]
# Ratio
para_t1t2_ratio=para_t1_sum/para_t2_sum
print(para_t1_sum,para_t2_sum,para_t1t2_ratio)

**Third step**: computation of subtraction image (<sup>99m</sup>Tc-sestamibi - 123 Iodine)

In [0]:
# Computation of subtraction image (99mTc-sestamibi - 123 Iodine)
# * 100 allows working with floats
substract=100*para_t1-100*para_t1t2_ratio*para_t2
print(substract.max())

plt.subplots(1,2,figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(substract,vmin=0,vmax=5000,aspect='auto')
plt.subplot(1,2,2)
plt.hist(substract.flatten(),256,[0,5000]); plt.show